In [12]:
"""writing Stockoption base class - reusable code for attributes of stock option
for this chapter"""

import math 

class StockOption(object):
    def __init__(self,S0,K,r=0.05,T=1,N=2,pu=0,pd=0,div=0,sigma=0,is_put=False, is_am=False) :
        """defaults to Euro call unless specified"""

        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.N = max(1,N) # number of time steps
        self.STs = [] # declare the stock prices tree
        
        self.pu, self.pd = pu, pd 
        self.div = div 
        self.sigma = sigma
        self.is_call = not is_put
        self.is_european = not is_am 
    
    @property
    def dt(self):
        return self.T/float(self.N)
        
        # the discount factor 
    @property
    def df(self):
        return math.exp(-(self.r-self.div)*self.dt)

In [13]:
import math 
import numpy as np
from decimal import Decimal

class BinomialTreeOption(StockOption):
    def setup_parameters(self):
    
        self.u = 1 + self.pu # expected value in up state
        self.d = 1-self.pd # expected value in down state
        self.qu = (math.exp((self.r-self.div)*self.dt)-self.d)/(self.u-self.d)
        self.qd = 1-self.qu
        
# store expected returns of stock price at all time steps in NumPy array

    def init_stock_price_Tree(self):
        self.STs = [np.array([self.S0])] # initialize 2D tree at T=0
        
        for i in range(self.N): # simulate possible stock prices path
            prev_branches = self.STs[-1]
            st = np.concatenate(
                (prev_branches*self.u,
                 [prev_branches[-1]*self.d]))
            self.STs.append(st) # add  nodes at each time step 

# creates payoff tree as 2D array. Starts with intrinsic option value @ maturity

    def init_payoff_tree(self):
        if self.is_call:
            return np.maximum(0,self.STs[self.N]-self.K)
        else:
            return np.maximum(0,self.K-self.STs[self.N])
  
 # returns maximum payoff values b/w exercising early or not at all
    def check_early_exercise(self,payoffs,node):
        if self.is_call:
            return np.maximum(payoffs, self.STs[node]-self.K)
        else:
            return np.maximum(payoffs,self.K-self.STs[node])
   
 # checks if optimal to exercise Am option early at every time step 
 # also moves through tree to find discount price at each step 
    def traverse_tree(self, payoffs):
        for i in reversed(range(self.N)):
            # payoff if not exercising the option
            payoffs = (payoffs[:-1]*self.qu + 
                       payoffs[1:]*self.qd)*self.df
            
            # payoff from exercising, for Am options
            if not self.is_european:
                payoffs = self.check_early_exercise(payoffs,i)
        
        return payoffs
    
    def begin_tree_traversal(self):
        payoffs = self.init_payoff_tree()
        return self.traverse_tree(payoffs)
    
    def price(self):
        """starting point of the tree and provides option value"""
        self.setup_parameters()
        self.init_stock_price_Tree()
        payoffs = self.begin_tree_traversal()
        
        return payoffs[0]

In [15]:
class BinomialLROption(BinomialTreeOption):
    def setup_parameters(self):
        odd_N = self.N if (self.N%2 == 0) else (self.N+1)           
        d1 = (math.log(self.S0/self.K)+((self.r-self.div)+
              (self.sigma**2)/2)*self.T)/(self.sigma*math.sqrt(self.T))   
        d2 = (math.log(self.S0/self.K)+((self.r-self.div)-
              (self.sigma**2)/2)*self.T)/(self.sigma * math.sqrt(self.T))
        pbar = self.pp_2_inversion(d1, odd_N)
        self.p = self.pp_2_inversion(d2, odd_N)
        self.u = (1/self.df) * (pbar/self.p)
        self.d = (1/self.df-self.p*self.u)/(1-self.p)
        self.qu = self.p
        self.qd = 1-self.p
        
    def pp_2_inversion(self, z, n):
        return .5 + math.copysign(1, z)*\
            math.sqrt(.25 - .25*math.exp(-((z/(n+1/3+.1/(n+1)))**2)*(n+1/6)))

In [17]:
eu_option = BinomialLROption(50,52,r=0.05,T=2,N=4,sigma=0.3,is_put=True)
print(eu_option.price())

5.878650106601972


In [18]:
"""Calculation of greeks for LR binomial model"""
import numpy as np

class BinomialLRGreeks(BinomialLROption):
    
    def new_stock_price_tree(self):
        """creates additional layer of nodes to original tree"""
        self.STs = [np.array([self.S0*self.u/self.d,self.S0,self.S0*self.d/self.u])]
        
        for i in range(self.N):
            prev_branches = self.STs[-1]
            st = np.concatenate((prev_branches*self.u,[prev_branches[-1]*self.d]))
            self.STs.append(st)
            
    def price(self):
        self.setup_parameters()
        self.new_stock_price_tree()
        payoffs = self.begin_tree_traversal()
        
        # option value now at middle node at t=0
        option_value = payoffs[len(payoffs)//2]
        
        payoff_up = payoffs[0]
        payoff_down = payoffs[-1]
        S_up = self.STs[0] [0]
        S_down = self.STs[0] [-1]
        dS_up = S_up - self.S0
        dS_down = self.S0 - S_down
        
        # calculate delta
        dS = S_up - S_down
        dV = payoff_up - payoff_down
        delta = dV/dS
        
        # calculate gamma 
        gamma = ((payoff_up-option_value)/dS_up -
                 (option_value-payoff_down)/dS_down)/ \
            ((self.S0+S_up)/2. - (self.S0+S_down)/2.)
        
        return option_value, delta, gamma 
    
    

In [20]:
eu_call =BinomialLRGreeks(50,52,r=0.05,T=2,N=300,sigma=0.3)
results = eu_call.price()
print('price: %s\nDelta: %s\nGamma: %s' % results )

price: 9.69546807138366
Delta: 0.6392477816643529
Gamma: 0.01764795890533088
